In [4]:
from datetime import date
import pandas as pd
from onnx.reference.ops.aionnxml.op_label_encoder import LabelEncoder

In [43]:
df = pd.read_csv('/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/ui/sample_tables/mvc_safety_rating_by_accidents.csv')
df

,Unnamed: 0,person_id,person_age,person_sex,vehicle_make,vehicle_year,overall_rating
0,0,6465de8a-bdb5-40dc-bf18-dd00535eca9c,53,F,honda,2003.0,4.166667
1,1,6465de8a-bdb5-40dc-bf18-dd00535eca9c,53,F,honda,2003.0,4.000000
2,2,7dd41360-6fef-4563-9292-d19ac7cf70b7,24,M,honda,2003.0,4.333333
3,3,7dd41360-6fef-4563-9292-d19ac7cf70b7,24,M,honda,2003.0,4.200000
4,4,f8abf7e4-4724-424b-afc3-49a50dd42390,24,M,honda,2003.0,4.166667
...,...,...,...,...,...,...,...
6561,6561,5e84e0d6-ab0d-4264-b344-b5dffc60f3dd,41,M,honda,2002.0,4.400000
6562,6562,026801b4-c6a6-414c-92ca-d81925156251,41,M,honda,2002.0,4.800000
6563,6563,90d01e14-8680-496c-bbd6-f7f8e4313550,69,F,dodge,2013.0,4.000000
6564,6564,10f43748-5177-4099-b2a6-7d69865940ba,46,F,jeep,2016.0,5.000000


In [44]:
df['vehicle_age'] = date.today().year - df['vehicle_year']
df['norm_overall_rating'] = df['overall_rating'] / 5

In [45]:
from sklearn.preprocessing import LabelEncoder
df['vehicle_label'] = LabelEncoder().fit_transform(df['vehicle_make'])

In [46]:
df[['vehicle_make', 'vehicle_label']]

,vehicle_make,vehicle_label
0,honda,7
1,honda,7
2,honda,7
3,honda,7
4,honda,7
...,...,...
6561,honda,7
6562,honda,7
6563,dodge,3
6564,jeep,8


In [47]:
df['vehicle_label'].unique()

array([ 7,  4,  0,  8,  9,  3,  6, 13,  2,  1, 10, 12, 11,  5])

In [50]:
from sklearn.manifold import SpectralEmbedding
embedding = SpectralEmbedding(n_components=4)
X_transformed = embedding.fit_transform(df['vehicle_label'].values.reshape(-1, 1))

In [51]:
X_transformed

array([[ 6.95221619e-04,  3.12778288e-05, -6.93806332e-04,
         7.79211637e-07],
       [ 6.95221619e-04,  3.12778288e-05, -6.93806332e-04,
         7.79211637e-07],
       [ 6.95221619e-04,  3.12778288e-05, -6.93806332e-04,
         7.79211637e-07],
       ...,
       [-3.21536587e-04, -2.56595521e-04,  2.47343520e-06,
         2.97363787e-03],
       [ 7.03258590e-04,  3.39943754e-05,  1.03976759e-03,
         4.74236183e-08],
       [ 7.03258590e-04,  3.39943754e-05,  1.03976759e-03,
         4.74236183e-08]])